In [1]:
%%init_spark
launcher.conf.spark.app.name = "archita_spark3"
launcher.conf.spark.local.dir = "/data/tmp/spark"
launcher.conf.spark.driver.memory = "100g"
launcher.conf.spark.executor.memory = "50g"
launcher.conf.spark.executors = "21"
launcher.conf.spark.executor.cores = "21"
launcher.conf.spark.sql.shuffle.partitions = 210
launcher.conf.spark.sql.shuffle.minPartitions = 20
launcher.conf.spark.ui.showConsoleProgress = "true"
launcher.conf.spark.yarn.keytab = "/home/jioapp/aditya/jiomart_cluster/couture.keytab"
launcher.conf.spark.yarn.principal = "couture@COUTURE.AI"
launcher.conf.spark.driver.extraJavaOptions = "-Djava.security.krb5.conf=/home/jioapp/aditya/jiomart_cluster/krb5.conf"
launcher.conf.spark.executor.extraJavaOptions = "-Djava.security.krb5.conf=/home/jioapp/aditya/jiomart_cluster/krb5.conf"

In [2]:
var df_tq = spark.read.parquet("hdfs://10.144.96.170:8020/data1/searchengine/testing/ajio/query_product_scored_top_100")

Intitializing Scala interpreter ...

Spark Web UI available at http://10.166.181.219:4048
SparkContext available as 'sc' (version = 3.3.0, master = local[*], app id = local-1739272042746)
SparkSession available as 'spark'


df_tq: org.apache.spark.sql.DataFrame = [query: string, products: array<struct<score:double,colorGroup_string:string>>]


In [3]:
df_tq.printSchema()

root
 |-- query: string (nullable = true)
 |-- products: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- score: double (nullable = true)
 |    |    |-- colorGroup_string: string (nullable = true)



In [4]:
df_tq.show()

+--------------------+--------------------+
|               query|            products|
+--------------------+--------------------+
|avaasa mix n&#39;...|[{0.7518140609999...|
|              avassa|[{0.7875362045664...|
|     black kurta set|[{0.7132429529390...|
|          black suit|[{1.1628902667258...|
|      crocs for boys|[{0.8881181879100...|
|  earrings for women|[{0.7666670444833...|
|         flats women|[{1.4732975056861...|
|  formal shirt women|[{1.5846966474848...|
|          grey jeans|[{1.4862912427823...|
|        jogger jeans|[{0.860187000445,...|
|   joggers for girls|[{0.6304325986932...|
|    kurta sets women|[{0.6174382458639...|
|                 lee|[{0.5325296184330...|
| loose fit jeans men|[{0.6012513714102...|
|     red chief shoes|[{0.7059111743190...|
|       shirt for boy|[{0.7949280300428...|
|              stoles|[{0.5807130449932...|
|      t shirt women |[{1.1253523057234...|
|     track pants men|[{0.7422959682151...|
|       women sliders|[{0.668177

In [5]:
df_tq = df_tq.withColumn("product",explode(col("products")))

df_tq: org.apache.spark.sql.DataFrame = [query: string, products: array<struct<score:double,colorGroup_string:string>> ... 1 more field]


In [6]:
df_tq = df_tq.drop("products")

df_tq: org.apache.spark.sql.DataFrame = [query: string, product: struct<score: double, colorGroup_string: string>]


In [7]:
df_tq = df_tq.select(col("query"),col("product.colorGroup_string").alias("id"),col("product.score").alias("score"))

df_tq: org.apache.spark.sql.DataFrame = [query: string, id: string ... 1 more field]


In [8]:
df_tq.printSchema()

root
 |-- query: string (nullable = true)
 |-- id: string (nullable = true)
 |-- score: double (nullable = true)



In [9]:
df_tq.show()

+--------------------+--------------------+------------------+
|               query|                  id|             score|
+--------------------+--------------------+------------------+
|avaasa mix n&#39;...|441144528_olivegreen|0.7518140609999999|
|avaasa mix n&#39;...|      441124685_blue|0.7420315622033999|
|avaasa mix n&#39;...|      441145550_blue|   0.7367807992086|
|avaasa mix n&#39;...|      441146701_teal|   0.7360080622821|
|avaasa mix n&#39;...|     441133434_peach|   0.7273994481226|
|avaasa mix n&#39;...|441149941_pistagreen|0.7266052330579408|
|avaasa mix n&#39;...|  441115970_offwhite|0.7249970551199432|
|avaasa mix n&#39;...|      443007662_pink|0.7234900550381574|
|avaasa mix n&#39;...|     441141761_coral|0.7226483943971731|
|avaasa mix n&#39;...|    443003252_ltpink| 0.721764438020237|
|avaasa mix n&#39;...| 441103249_limegreen|0.7215667576602801|
|avaasa mix n&#39;...|     441124214_black|0.7195380612258001|
|avaasa mix n&#39;...|      441134610_grey|  0.71755374

In [10]:
var df_tq_2 = spark.read.parquet("hdfs://10.144.96.170:8020/data1/searchengine/processedHistory20240301/ajio/NormalizedNonZeroProductInteractionsCombined")

df_tq_2: org.apache.spark.sql.DataFrame = [date: string, search_type: string ... 8 more fields]


In [11]:
df_tq_2.printSchema()

root
 |-- date: string (nullable = true)
 |-- search_type: string (nullable = true)
 |-- search_term: string (nullable = true)
 |-- option_code: string (nullable = true)
 |-- unique_session_seen: long (nullable = true)
 |-- impressions: long (nullable = true)
 |-- clicks: long (nullable = true)
 |-- total_revenue: double (nullable = true)
 |-- total_quantity: long (nullable = true)
 |-- query_normalised: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [19]:
df_tq_2.filter(col("clicks")>col("impressions")).count()

res8: Long = 44675488


In [12]:
df_tq_2 = df_tq_2.withColumn("query",concat_ws(" ",col("query_normalised")))

df_tq_2: org.apache.spark.sql.DataFrame = [date: string, search_type: string ... 9 more fields]


In [13]:
df_tq_2 = df_tq_2.groupBy("query","option_code")
.agg(
    sum("impressions").alias("impressions"),
    sum("clicks").alias("clicks"),
).withColumn(
    "ctr",col("clicks")/col("impressions")
)

df_tq_2: org.apache.spark.sql.DataFrame = [query: string, option_code: string ... 3 more fields]


In [14]:
df_tq_2.write.option("header",true).parquet("file:////app/notebooks/avinash/CTR/temp_dir/temp.parquet")

In [16]:
var df_tq_3 = spark.read.option("header",true).parquet("file:////app/notebooks/avinash/CTR/temp_dir/temp.parquet")

df_tq_3: org.apache.spark.sql.DataFrame = [query: string, option_code: string ... 3 more fields]


In [17]:
df_tq_3.show()

+--------------------+--------------------+-----------+------+--------------------+
|               query|         option_code|impressions|clicks|                 ctr|
+--------------------+--------------------+-----------+------+--------------------+
|       socks for men|469503742_carbonb...|         54|     8| 0.14814814814814814|
|                caps|       442125070_red|       2083|    87|0.041766682669227076|
|      jeans jeggings|     465249662_white|     173443|  4965|0.028626119243786143|
|gold colour plane...|      461488344_blue|          1|     1|                 1.0|
|     shirt for women|      443009454_pink|        931|    85| 0.09129967776584318|
|              kurtas|     443007682_black|     925745| 18884|0.020398705907134253|
|            handbags|     469568443_black|       2644|   119|0.045007564296520426|
|             kimaira|      464969996_teal|          8|     6|                0.75|
|              svrnaa|   441213450_mustard|       4676|   398| 0.08511548331

In [18]:
df_tq_3.filter(col("clicks")>col("impressions")).count()

res7: Long = 44675488


In [12]:
var joined_df = df_tq.join(df_tq_3,df_tq("query")===df_tq_3("query") && df_tq("id")===df_tq_3("option_code"),"left_outer")
.drop(
    df_tq_3("query")
).drop(df_tq_3("impressions")).drop(df_tq_3("clicks")).drop(df_tq_3("option_code"))

joined_df: org.apache.spark.sql.DataFrame = [query: string, id: string ... 2 more fields]


In [13]:
joined_df.show()

+--------------------+--------------------+------------------+--------------------+
|               query|                  id|             score|                 ctr|
+--------------------+--------------------+------------------+--------------------+
|    aavasa kurta set|      440960737_blue|  0.63124241194602|                null|
|      american eagle|      460982167_blue| 0.603387042340285|                 0.1|
|              avaasa|      441177985_blue|0.6508442145969531|                null|
|               bikni|     462362440_green|0.6295511720236723|                null|
|  anarkali for women|    464854096_purple|0.5675540506766841|0.021047596268835206|
|      anarkali kurti|    464363602_yellow|0.5804269032123549|                 1.0|
|          adidas cap|     410264726_black|0.5986362396783556| 0.02685325264750378|
|                acai|     469228247_green|0.5504899508635408|  0.1136789506558401|
|                acai|  469228258_seagreen|0.5583710281594695| 0.05393482282

In [14]:
// joined_df.write.option("header",true).parquet("file:////app/notebooks/avinash/CTR/CTR-output.parquet")

In [15]:
var null_ctr_df=joined_df.filter(col("ctr").isNull)

null_ctr_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [query: string, id: string ... 2 more fields]


In [16]:
null_ctr_df.show()

+--------------------+--------------------+-------------------+----+
|               query|                  id|              score| ctr|
+--------------------+--------------------+-------------------+----+
|                   a|4928740550_sweeta...|0.33627840254999997|null|
|             aavaasa|   441074603_mustard|    0.5903793948572|null|
|        aavasa kurta|      441035132_aqua|   0.68002993099217|null|
|    aavasa kurta set|      440960737_blue|   0.63124241194602|null|
|    aavasa kurta set|  441024037_ltorange|    0.5965028142255|null|
|       aavasa kurtas|      441055704_navy| 0.6779514591821161|null|
|      addidas shoes |     469107522_black|   0.68105724570706|null|
|     adidas backpack|     460776866_multi|         0.57385194|null|
|       adidas terrex|     460930503_black|   0.53607642294164|null|
|       adidas tshirt|     469111891_green|   0.69800189916795|null|
|             adizero|     460929979_black|0.42664571135167867|null|
|              airmax|     4580051

In [17]:
null_ctr_df.count()

res8: Long = 195549


In [18]:
joined_df.count()

res9: Long = 468964


In [19]:
joined_df.filter(col("query") === "mixer").show()

+-----+------------------+-------------------+-----------------+
|query|                id|              score|              ctr|
+-----+------------------+-------------------+-----------------+
|mixer|   410059722_noclr|     0.382862983475|             null|
|mixer|   410059828_noclr|0.39605933984880004|             null|
|mixer|   410059869_noclr|     0.374602575105|             null|
|mixer|410061328_nocolour|0.41666000021618405|             null|
|mixer| 410065874_nocolor|     0.378054904776|             null|
|mixer| 410065875_nocolor|0.38882983790699993|             null|
|mixer| 410065877_nocolor|    0.3776055625908|             null|
|mixer|     410070436_etc|          0.3782782|             null|
|mixer|     410079628_etc|    0.3982562452255|             null|
|mixer|     410081113_etc|    0.3833394109024|5.333333333333333|
|mixer|     410242110_091|0.39673555768319996|             null|
|mixer|     410242111_091|    0.3922935386963|             null|
|mixer|     410244790_000

In [17]:
var not_null_df = joined_df.filter(!col("ctr").isNull)

not_null_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [query: string, id: string ... 2 more fields]


In [18]:
not_null_df = not_null_df.groupBy("query").agg(avg(col("ctr")).alias("avg_ctr_query_level"))

not_null_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [query: string, avg_ctr_query_level: double]


In [19]:
not_null_df = not_null_df.orderBy(desc("avg_ctr_query_level"))

not_null_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [query: string, avg_ctr_query_level: double]


In [20]:
not_null_df.show()

25/02/11 16:21:18 WARN SparkConf: The configuration key 'spark.yarn.keytab' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.keytab' instead.
25/02/11 16:21:18 WARN SparkConf: The configuration key 'spark.yarn.principal' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.principal' instead.
+--------------------+-------------------+
|               query|avg_ctr_query_level|
+--------------------+-------------------+
|               mixer|  5.333333333333333|
|      football studs|                2.0|
|              fleece|                2.0|
|       legenga choli| 1.5569948186528497|
|            sherwani|             1.3125|
|          tunic tops|               1.25|
|      downshifter 13| 1.2047619047619047|
|      jean for women| 1.1164625850340137|
|           coard set| 1.0833333333333333|
|  varsity jacket men| 1.0801617143555882|
|                 boy| 1.038256

In [21]:
not_null_df = not_null_df.withColumn("avg_ctr_query_level",when(col("avg_ctr_query_level")>1,1).otherwise(col("avg_ctr_query_level")))

not_null_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [query: string, avg_ctr_query_level: double]


In [23]:
not_null_df.show()

25/02/11 16:22:19 WARN SparkConf: The configuration key 'spark.yarn.keytab' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.keytab' instead.
25/02/11 16:22:19 WARN SparkConf: The configuration key 'spark.yarn.principal' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.principal' instead.
+--------------------+-------------------+
|               query|avg_ctr_query_level|
+--------------------+-------------------+
|               mixer|                1.0|
|      football studs|                1.0|
|              fleece|                1.0|
|       legenga choli|                1.0|
|            sherwani|                1.0|
|          tunic tops|                1.0|
|      downshifter 13|                1.0|
|      jean for women|                1.0|
|           coard set|                1.0|
|  varsity jacket men|                1.0|
|                 boy|         

In [22]:
not_null_df.write.option("header",true).parquet("hdfs://10.144.96.170:8020/data1/archive/avinash/SearchTests/CTR/ctr-query-level.parquet")

25/02/11 16:21:46 WARN SparkConf: The configuration key 'spark.yarn.keytab' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.keytab' instead.
25/02/11 16:21:46 WARN SparkConf: The configuration key 'spark.yarn.principal' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.principal' instead.


In [ ]:
not_null_df.write.option("header",true).parquet("/data1/archive/avinash/SearchTests/CTR/ctr-query-level.parquet")

25/02/11 16:33:38 WARN UserGroupInformation: Not attempting to re-login since the last re-login was attempted less than 60 seconds before. Last Login=1739271814605
25/02/11 16:33:42 WARN UserGroupInformation: Not attempting to re-login since the last re-login was attempted less than 60 seconds before. Last Login=1739271814605
25/02/11 16:33:47 WARN UserGroupInformation: Not attempting to re-login since the last re-login was attempted less than 60 seconds before. Last Login=1739271814605
25/02/11 16:33:51 WARN UserGroupInformation: Not attempting to re-login since the last re-login was attempted less than 60 seconds before. Last Login=1739271814605
25/02/11 16:33:55 WARN Client: Couldn't setup connection for couture@COUTURE.AI to jmngd1baf370c10/10.144.97.194:8020
javax.security.sasl.SaslException: GSS initiate failed [Caused by GSSException: No valid credentials provided (Mechanism level: Server not found in Kerberos database (7) - LOOKING_UP_SERVER)]
	at com.sun.security.sasl.gsskerb.